In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0.1)

class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")

#OPENAI FUNCTION
class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b

agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool()
])
prompt = "Cost of $325.34 + $12.32 + $999.23 + $23843.34"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 325.34, 'b': 12.32}`


337.65999999999997
Invoking: `CalculatorTool` with `{'a': 337.66, 'b': 999.23}`


1336.89
Invoking: `CalculatorTool` with `{'a': 1336.89, 'b': 23843.34}`


25180.23The total cost of $325.34 + $12.32 + $999.23 + $23843.34 is $25,180.23.

> Finished chain.


{'input': 'Cost of $325.34 + $12.32 + $999.23 + $23843.34',
 'output': 'The total cost of $325.34 + $12.32 + $999.23 + $23843.34 is $25,180.23.'}